In [1]:
import pandas as pd
import plotly.express as px

In [7]:
INPUTS = ["PC", "Controller"]
MAPS = ["all-maps"]  # Can be extended with specific map names
REGIONS = ["Europe", "US", "Asia"]
ROLES = ["All", "Damage", "Tank", "Support"]
RQ_OPTIONS = [0, 1]  # Role queue flag
TIERS = ["All", "Bronze", "Silver", "Gold", "Platinum", "Diamond", "Master", "Grandmaster"]

def load_stats_data(input_type="PC", map_name="all-maps", region="Europe", role="All", rq=1, tier="All"):
    """
    Load Overwatch 2 hero statistics data from a CSV file based on specified parameters.
    
    Args:
        input_type (str): Input device type ("PC" or "Controller")
        map_name (str): Map name or "all-maps"
        region (str): Region ("Europe", "US", or "Asia")
        role (str): Hero role ("All", "Damage", "Tank", or "Support")
        rq (int): Role queue flag (0 or 1)
        tier (str): Competitive tier ("All", "Bronze", "Silver", "Gold", "Platinum", "Diamond", "Master", "Grandmaster")
    
    Returns:
        pandas.DataFrame: DataFrame containing the hero statistics
    """
    # Construct the filename based on parameters
    filename = f"stats_input-{input_type.lower()}_map-{map_name.lower()}_region-{region.lower()}_role-{role.lower()}_rq-{rq}_tier-{tier.lower()}.csv"
    filepath = f"data/{filename}"
    
    try:
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(filepath)
        return df
    except FileNotFoundError:
        print(f"Error: File not found - {filepath}")
        return None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None


In [40]:
winrate_per_tier = {} # {hero: {tier: winrate}}

for tier in TIERS:
    print(f"Loading data for tier: {tier}")
    df = load_stats_data(tier=tier)
    for hero in df["id"].tolist():
        print(f"- Loading data for hero: {hero}")
        hero_df = df[df["id"] == hero]
        winrate_per_tier.setdefault(hero, {})
        winrate_per_tier[hero][tier] = {
            "winrate": hero_df["cells_winrate"].mean(),
            "pick_rate": hero_df["cells_pickrate"].mean(),
            "hero_name": hero_df["hero_name"].iloc[0],
            "hero_color": "#" + hero_df["hero_color"].iloc[0].upper()[:6]
        }
        print(winrate_per_tier[hero][tier])


Loading data for tier: All
- Loading data for hero: ana
{'winrate': 48.5, 'pick_rate': 29.9, 'hero_name': 'Ana', 'hero_color': '#48699E'}
- Loading data for hero: ashe
{'winrate': 50.2, 'pick_rate': 12.4, 'hero_name': 'Ashe', 'hero_color': '#3E3C3A'}
- Loading data for hero: baptiste
{'winrate': 46.9, 'pick_rate': 9.2, 'hero_name': 'Baptiste', 'hero_color': '#28A5C3'}
- Loading data for hero: bastion
{'winrate': 46.5, 'pick_rate': 10.1, 'hero_name': 'Bastion', 'hero_color': '#5B7351'}
- Loading data for hero: brigitte
{'winrate': 47.8, 'pick_rate': 3.7, 'hero_name': 'Brigitte', 'hero_color': '#72332A'}
- Loading data for hero: cassidy
{'winrate': 47.9, 'pick_rate': 30.2, 'hero_name': 'Cassidy', 'hero_color': '#A62927'}
- Loading data for hero: dva
{'winrate': 48.5, 'pick_rate': 10.5, 'hero_name': 'D.Va', 'hero_color': '#FC79BD'}
- Loading data for hero: doomfist
{'winrate': 54.6, 'pick_rate': 7.8, 'hero_name': 'Doomfist', 'hero_color': '#661E0F'}
- Loading data for hero: echo
{'winrate

In [41]:
# Get the list of heroes and tiers
heroes = list(winrate_per_tier.keys())
tiers = [tier for tier in TIERS if tier != "All"]  # Exclude "All" for better visualization

# Create a figure
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# Calculate grid dimensions for subplots
n_heroes = len(heroes)
n_cols = 3  # You can adjust this
n_rows = math.ceil(n_heroes / n_cols)

# Create subplots
fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=heroes)

# Add traces for each hero
for i, hero in enumerate(heroes):
    row = i // n_cols + 1
    col = i % n_cols + 1
    
    # Extract winrates for this hero across tiers
    winrates = []
    tier_indices = []
    
    # Get hero color - use the first available tier's color
    hero_color = None
    for tier in tiers:
        if tier in winrate_per_tier[hero]:
            hero_color = winrate_per_tier[hero][tier]["hero_color"]
            # Remove the alpha channel if present (last two characters in hex color)
            if hero_color and len(hero_color) == 9:  # Format: #RRGGBBAA
                hero_color = hero_color[:7]  # Keep only #RRGGBB
            break
    
    for idx, tier in enumerate(tiers):
        if tier in winrate_per_tier[hero]:
            winrates.append(winrate_per_tier[hero][tier]["winrate"])
            tier_indices.append(idx)  # Use indices for x-values
        else:
            print(f"Error: Tier {tier} not found for hero {hero}")
            # Skip missing data points instead of adding None

    # Add trace to the appropriate subplot
    fig.add_trace(
        go.Scatter(
            x=tier_indices,  # Use indices for x-values to match tickvals
            y=winrates,
            mode='lines+markers',
            name=hero,
            showlegend=False,
            line=dict(width=2, color=hero_color),  # Use hero color for the line
            marker=dict(color=hero_color),  # Use hero color for markers
            connectgaps=True     # Connect gaps caused by None values
        ),
        row=row, col=col
    )
for i in range(n_heroes):
    row = i // n_cols + 1
    col = i % n_cols + 1
    # Update x-axis to show tier names
    fig.update_xaxes(tickvals=list(range(len(tiers))), ticktext=tiers, row=row, col=col)
    
    # Update y-axis range to focus on relevant winrate range (e.g., 20-80%)
    fig.update_yaxes(range=[20, 80], row=row, col=col)

# Update layout
fig.update_layout(
    title_text="Winrate by Tier for Each Hero",
    height=300 * n_rows,  # Adjust height based on number of rows
    width=1200,
    template="plotly_white"
)

# Show the figure
fig.show()

# Create a single plot with all heroes for comparison
fig_combined = go.Figure()

# Add a trace for each hero
for hero in heroes:
    winrates = []
    hero_color = None
    for tier in tiers:
        if tier in winrate_per_tier[hero]:
            winrates.append(winrate_per_tier[hero][tier]["winrate"])
            hero_color = winrate_per_tier[hero][tier]["hero_color"]
        else:
            winrates.append(None)
    
    fig_combined.add_trace(
        go.Scatter(
            x=tiers,
            y=winrates,
            mode='lines+markers',
            name=hero,
            line=dict(width=2, color=hero_color),  # Use hero color for the line
            visible="legendonly" if hero != heroes[0] else True
        )
    )

# Update layout
fig_combined.update_layout(
    title="Comparison of Hero Winrates Across Tiers",
    xaxis_title="Tier",
    yaxis_title="Winrate",
    height=800,
    width=1200,
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.5,
        xanchor="center",
        x=0.5
    ),
    yaxis_range=[20, 80]
)

# Show the figure
fig_combined.show()
